## Matrix and Vocabulary Construction

In [4]:
import pandas as pd

import numpy as np

from scipy import sparse

import nltk
from nltk import bigrams
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/joao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/joao/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
news = pd.read_csv("../Lab3/data/estadao_noticias_eleicao.csv",encoding="utf-8")
news = news.fillna("")

In [6]:
content = news.titulo + " " + news.subTitulo + " " + news.conteudo
content = content.fillna("")

In [7]:
def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

#### Removing punctuation

In [8]:
tokenizer = RegexpTokenizer(r'\w+')
tokens_lists = content.apply(lambda text: tokenizer.tokenize(text.lower()))

#### Removing stopwords

In [9]:
stopword_ = stopwords.words('portuguese')
filtered_tokens = tokens_lists.apply(lambda tokens: [token for token in tokens if token not in stopword_])

#### Transforming list of lists into one list

In [10]:
tokens = [token for tokens_list in filtered_tokens for token in tokens_list]

In [11]:
matrix, vocab = co_occurrence_matrix(tokens)

In [12]:
def top3_words(text):
    cont = 0
    docs = []
    for i, j, k in zip(matrix.row, matrix.col, matrix.data):
        if(cont == 3):
            break
        if(vocab[text] == i):
            cont = cont + 1
            docs.append(((list(vocab.keys())[list(vocab.values()).index(j)])))
                   
    return docs

In [13]:
id_news = dict()

In [14]:
def search_term(palavra_pesquisada):
    if(palavra_pesquisada in id_news):
        return id_news[palavra_pesquisada]
    else:
        for x in range(news.shape[0]):             #fazer disso uma funcao  usar -> df.shape[0]  
            titulo = news.loc[x,'titulo']         #pego o titulo df.loc -> provavelmente no lab será df.ix
            conteudo = news.loc[x,'conteudo']  #pego o conteudo
            sub_titulo = news.loc[x,'subTitulo']
            newWorld = titulo +  " " +  conteudo + " " + sub_titulo      #concateno os tres separado por espaço
            newNova = nltk.word_tokenize(newWorld.lower())   #faço split usando nltk botando tudo para lower case
            for y in range(len(newNova)):               #faço outro for para percorrer esse novo split procurando a palavra que eu quero
                if(newNova[y] == palavra_pesquisada):  
                    if(palavra_pesquisada in id_news):
                        id_news[palavra_pesquisada].append(news.loc[x,'idNoticia'])              #posso usar o x do primeiro for por que é o mesmo número do ID
                        break;                        #parar para que eu não coloque mais do mesmo indice, caso apareça mais de uma vez
                    else:
                        id_news[palavra_pesquisada] = [news.loc[x,'idNoticia']]
                        break;    
                    
        return id_news[palavra_pesquisada]

In [15]:
def search_for_tops(term):
    tops = top3_words(term)
    tops.append(term)
    result = set()
    for i in tops:
        print(i)
        aux = set(search_term(i))
        result = result.union(aux)
    return result

<h3>Análise do Algoritmo</h3>

Em relação a precisão e recall, teremos um trade-off quando compararmos os resultados. Para essa análise, vamos usar os resultados da pesquisa 'lula'

In [16]:
len(search_for_tops('lula'))

silva
dilma
disse
lula


6402

    Vemos que, quando pesquisamos a pesquisa expandida(nesse caso teremos as palavras: SILVA,DILMA,DISSE) o número de documentos retornados vai ser muito maior, ou seja, o recall dessa pesquisa será muito maior do que a pesquisa só de LULA, deixando sua precisão baixa. Porém, temos uma coisa que conta ao favor dessa consulta expandida. Pessoas que pesquisam por LULA podem tender a pesquisar por DILMA também, ou seja, o retorno desses documentos pode ser o mais esperado pelo usuário.

In [17]:
len(search_term('lula'))

1891

     Já quando pesquisamos apenas por LULA, vamos o trade-off acontecendo. A precisão será maior, dado que o número de documentos retornados será muito menor, afetando também o recall, sendo menor nesse segundo caso.

## Consult Bigram Frequency

In [18]:
consultable_matrix = matrix.tocsr()

In [19]:
def consult_frequency(w1, w2):
    return(consultable_matrix[vocab[w1],vocab[w2]])

#### Example

In [20]:
w1 = 'poucos'
w2 = 'recursos'
consult_frequency(w1, w2)

3